In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import ConvLSTM2D
from pandas import Series
import pandas as pd
import numpy as np

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [3]:
# define input sequence
#raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130]

columns = ['steps']
df = pd.read_csv(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\p01_merged_multi.csv')
df = df[columns]

raw_seq = df.values.flatten()

In [4]:
# choose a number of time steps
n_steps_1 = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps_1)

In [5]:
# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))

In [6]:
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [7]:
# fit model
model.fit(X, y, epochs=500, verbose=0)

In [8]:
# demonstrate prediction
#x_input = array([100, 110, 120, 130])

x_input = df[-n_steps_1:]
x_input = x_input.values.flatten()

print(x_input)

x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[16448  6698  4761  9351]
[[10453.191]]
